In [4]:
import pandas as pd
import numpy as np
import geopandas as gpd


import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
tracts_new = gpd.read_file('jc_tract.geojson')

DriverError: Failed to read GeoJSON data

In [3]:
tracts_new.head()

,gid,statefp,countyfp,tractce,geoid,name,namelsad,mtfcc,funcstat,aland,awater,intptlat,intptlon,geometry
0,42214,34,017,005500,34017005500,55,Census Tract 55,G5020,S,216661,0,+40.7043624,-074.0834034,"POLYGON ((-74.08731 40.70388, -74.08663 40.704..."
1,42066,34,017,006600,34017006600,66,Census Tract 66,G5020,S,344312,0,+40.7248851,-074.0583365,"POLYGON ((-74.06203 40.72660, -74.06152 40.727..."
2,42093,34,017,007200,34017007200,72,Census Tract 72,G5020,S,296337,0,+40.7289059,-074.0480270,"POLYGON ((-74.05390 40.72759, -74.05385 40.727..."
3,42207,34,017,002300,34017002300,23,Census Tract 23,G5020,S,144800,0,+40.7235905,-074.0476910,"POLYGON ((-74.04937 40.72222, -74.04937 40.722..."
4,40796,34,017,002400,34017002400,24,Census Tract 24,G5020,S,159731,0,+40.7265797,-074.0452695,"POLYGON ((-74.04724 40.72493, -74.04717 40.725..."


In [8]:
tracts = gpd.read_file('censustract.geojson')

In [9]:
tracts.head()

,gid,statefp,countyfp,tractce,geoid,name,namelsad,mtfcc,funcstat,aland,awater,intptlat,intptlon,geometry
0,41576,34,017,005200,34017005200,52,Census Tract 52,G5020,S,376346.0,0.0,+40.7089087,-074.0852230,"MULTIPOLYGON (((-74.09265 40.71166, -74.09227 ..."
1,40796,34,017,002400,34017002400,24,Census Tract 24,G5020,S,159731.0,0.0,+40.7265797,-074.0452695,"MULTIPOLYGON (((-74.04724 40.72493, -74.04717 ..."
2,40800,34,017,005600,34017005600,56,Census Tract 56,G5020,S,231843.0,0.0,+40.7059678,-074.0913621,"MULTIPOLYGON (((-74.09586 40.70873, -74.09532 ..."
3,40801,34,017,000902,34017000902,9.02,Census Tract 9.02,G5020,S,521914.0,0.0,+40.7399426,-074.0623860,"MULTIPOLYGON (((-74.06646 40.73635, -74.06641 ..."
4,40802,34,017,004300,34017004300,43,Census Tract 43,G5020,S,220217.0,0.0,+40.7164541,-074.0709263,"MULTIPOLYGON (((-74.07566 40.71569, -74.07483 ..."


In [10]:
tracts['geometry'] = tracts['geometry'].astype(str)

In [11]:
tracts['geometry'] = tracts['geometry'].str[16:-3]

In [12]:
tracts['geometry'] = tracts['geometry'].astype(str)

In [13]:
tracts['geometry'] = tracts['geometry'].str.split(',')

In [14]:
tracts['geometry'].loc[0][1].split(' ')[1:]

['-74.09227300000001', '40.711896']

In [15]:
def centerpoints_lng(x):
    target_ = x.copy()
    for i in range(len(x)):
        if i == 0:
            target_[i] = target_[i].split(' ')
        else:
            target_[i] = target_[i].split(' ')[1:]
    target_ = np.array(target_)
    for i in range(len(target_)):
        try:
            target_[i] = np.array(target_[i]).astype(float)
        except:
            target_[i] = np.array([np.nan,np.nan])
    target_ = target_[:,0].astype(float)
    center_lng = (max(target_) + min(target_))/2
    return center_lng

In [16]:
def centerpoints_lat(x):
    target_ = x.copy()
    for i in range(len(x)):
        if i == 0:
            target_[i] = target_[i].split(' ')
        else:
            target_[i] = target_[i].split(' ')[1:]
    target_ = np.array(target_)
    for i in range(len(target_)):
        try:
            target_[i] = np.array(target_[i]).astype(float)
        except:
            target_[i] = np.array([np.nan,np.nan])
    target_ = target_[:,1].astype(float)
    center_lng = (max(target_) + min(target_))/2
    return center_lng

In [17]:
tracts['center_lng'] = tracts['geometry'].apply(lambda x:centerpoints_lng(x))

In [18]:
tracts['center_lat'] = tracts['geometry'].apply(lambda x:centerpoints_lat(x))

In [19]:
tracts = tracts.drop('geometry',axis=1)

In [20]:
tracts = pd.DataFrame(tracts)

In [23]:
tracts = tracts[['name','center_lng','center_lat']]

In [24]:
tracts = tracts.set_index('name')

In [25]:
tracts

,center_lng,center_lat
name,,
52,-74.085396,40.708846
24,-74.045277,40.726574
56,-74.091518,40.705998
9.02,-74.062474,40.741328
43,-74.071393,40.716794
...,...,...
72,-74.047732,40.728519
73,-74.047275,40.713518
23,-74.047693,40.723929


In [28]:
tracts.to_json('centerpoints.json',orient='index')

In [26]:
score_informations = pd.read_csv('jc_tract_scores_nov30.csv')

In [28]:
score_informations = score_informations.drop('tract_id',axis=1)

In [30]:
score_informations['census_name'] = score_informations['census_name'].str.strip('Census Tract ')

In [32]:
score_informations.head()

,census_name,imp_percent,imp_score,flood_score,green_score,greenview_index,storwater_managed,treecountestimate,overall_score
0,55,89.419004,1,3,0,1.000000,0,0,1.333333
1,66,85.712628,1,3,5,23.359663,432000,540,3.000000
2,72,67.826788,2,0,5,21.001900,417600,522,2.333333
3,23,91.552611,0,3,0,1.000000,0,0,1.000000
4,24,79.873525,1,3,5,27.624349,73600,92,3.000000


In [34]:
tract_information = score_informations

In [37]:
tract_information['center_lng']= tract_information['census_name'].apply(lambda x: tracts['center_lng'].loc[x])

In [44]:
tract_information['center_lat']= tract_information['census_name'].apply(lambda x: tracts['center_lat'].loc[x])

,imp_percent,imp_score,flood_score,green_score,greenview_index,storwater_managed,treecountestimate,overall_score,center_lng,center_lat
census_name,,,,,,,,,,
55,89.419004,1,3,0,1.000000,0,0,1.333333,-74.083714,40.704454
66,85.712628,1,3,5,23.359663,432000,540,3.000000,-74.058198,40.725267
72,67.826788,2,0,5,21.001900,417600,522,2.333333,-74.047732,40.728519
23,91.552611,0,3,0,1.000000,0,0,1.000000,-74.047693,40.723929
24,79.873525,1,3,5,27.624349,73600,92,3.000000,-74.045277,40.726574
...,...,...,...,...,...,...,...,...,...,...
58.01,76.867946,2,1,5,16.779130,1444800,1806,2.666667,-74.082866,40.699299
45,81.918562,1,1,0,15.305738,627200,784,0.666667,-74.072220,40.711020
9.02,81.725338,1,0,0,14.257763,616000,770,0.333333,-74.062474,40.741328


In [50]:
tract_information = tract_information.set_index('census_name')

In [51]:
tract_information.to_json('tract_info.json',orient='index')

In [49]:
tract_information

,census_name,imp_percent,imp_score,flood_score,green_score,greenview_index,storwater_managed,treecountestimate,overall_score,center_lng,center_lat
0,55,89.419004,1,3,0,1.000000,0,0,1.333333,-74.083714,40.704454
1,66,85.712628,1,3,5,23.359663,432000,540,3.000000,-74.058198,40.725267
2,72,67.826788,2,0,5,21.001900,417600,522,2.333333,-74.047732,40.728519
3,23,91.552611,0,3,0,1.000000,0,0,1.000000,-74.047693,40.723929
4,24,79.873525,1,3,5,27.624349,73600,92,3.000000,-74.045277,40.726574
...,...,...,...,...,...,...,...,...,...,...,...
57,58.01,76.867946,2,1,5,16.779130,1444800,1806,2.666667,-74.082866,40.699299
58,45,81.918562,1,1,0,15.305738,627200,784,0.666667,-74.072220,40.711020
59,9.02,81.725338,1,0,0,14.257763,616000,770,0.333333,-74.062474,40.741328
60,17.01,82.431279,1,1,3,13.599446,728000,910,1.666667,-74.077293,40.736269
